In [1]:
from pyspark.sql.types import IntegerType
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *

import sys

sys.path.append("..")  # Adjust path if needed
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *

from pyspark.sql.functions import col, lit, max as spark_max, length, row_number, explode

def create_spark_local_session():
    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [2]:
spark = create_spark_local_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
#delta_news_tmdb = "../data/delta_tmbd"
#delta_tmdb = "../data/delta_news/entertainment"

In [75]:
absolute_path_to_landing = r'C:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\data\letstalk_landing_zone_bdma'
table_subpaths = [r'delta_tmdb\database\movie',r'delta_tmdb\now_playing',r'delta_tmdb\trending',r'delta_tmdb\upcoming']
i=3
loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpaths[i])
df = loader.get_new_data()
print(df.show(10))

CDF not available — doing full load
+-----+--------------------+------------------+-------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+--------------------+----------+----------+
|adult|       backdrop_path|         genre_ids|     id|original_language|      original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|      ingestion_time|begin_date|  end_date|
+-----+--------------------+------------------+-------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+--------------------+----------+----------+
|false|/op3qmNhvwEvyT7UF...|      [28, 14, 12]| 324544|               en|   In the Lost Lands|A queen sends the...|  307.5934|/twy1IVJFgaZWA21S...|  2025-02-27|   In the Lost Lands|false|       6.325

In [86]:
processor = TMDBProcessor(spark, df)
processor.ensure_schema()
processor.remove_clear_duplicates()
processor.normalize_text(['overview'])

processor.df

#processor.normalize_text(['title', 'description', 'content'])
#processor.order_by('publishedAt', ascending=False)


INFO:root:Removed 0 simple duplicate(s)


adult,backdrop_path,genre_ids,film_id,original_language,title,overview,popularity,poster_path,release_date,delete,video,vote_average,vote_count,ingestion_time,begin_date,end_date,id,budget,revenue,runtime
false,/eDBZN0TbWkxoAB0q...,"[27, 14, 35, 12]",1153714,en,Death of a Unicorn,a father and daug...,290.5672,/lXR32JepFwD1UHkp...,2025-03-27,Death of a Unicorn,false,6.253,146,2025-05-05 23:08:...,2025-05-07,2025-05-28,1,0,0,0
false,/op3qmNhvwEvyT7UF...,"[28, 14, 12]",324544,en,In the Lost Lands,a queen sends the...,265.4894,/dDlfjR7gllmr8HTe...,2025-02-27,In the Lost Lands,false,6.345,330,2025-05-05 23:08:...,2025-05-07,2025-05-28,2,0,0,0
false,/12tEzU0bNYKIjXXE...,[37],710258,en,Rust,infamous outlaw h...,210.9704,/tbJ3RkA2s6X5qrBz...,2025-05-01,Rust,false,6.8,17,2025-05-05 23:08:...,2025-05-07,2025-05-28,3,0,0,0
false,/nAxGnGHOsfzufThz...,"[27, 10402, 53]",1233413,en,Sinners,trying to leave t...,202.7984,/jYfMTSiFFK7ffbY2...,2025-04-16,Sinners,false,7.615,553,2025-05-05 23:08:...,2025-05-07,2025-05-28,4,0,0,0
false,/bFmziv84NfEVEGTK...,"[28, 80, 9648, 53]",1180906,en,Desert Dawn,a newly appointed...,189.4444,/S21BfLrJSD9njucB...,2025-05-15,Desert Dawn,false,0.0,0,2025-05-05 23:08:...,2025-05-07,2025-05-28,5,0,0,0
false,/rdxdvXQGqCj2Vb7S...,"[80, 53, 28]",870028,en,The Accountant²,when an old acqua...,108.5178,/kMDUS7VmFhb2coRf...,2025-04-23,The Accountant²,false,7.2,114,2025-05-05 23:08:...,2025-05-07,2025-05-28,6,0,0,0
false,/3lEV4CoKoeT2cZ4f...,"[27, 9648, 53]",1244944,en,The Woman in the ...,in the aftermath ...,80.6427,/n0WS2TsNcS6dtaZK...,2025-03-27,The Woman in the ...,false,6.2,131,2025-05-05 23:08:...,2025-05-07,2025-05-28,7,0,0,0
false,/kXxriYfAWtMJNHGH...,"[53, 878]",650033,ru,Свободное падение,in the near futur...,77.5824,/tB66c6dnu9dG60mt...,2025-05-14,Free Fall,false,0.0,0,2025-05-05 23:08:...,2025-05-07,2025-05-28,8,0,0,0
false,/hwlyY7LJdEFbCPaG...,"[28, 53]",1126166,en,Flight Risk,a u s marshal es...,71.673,/q0bCG4NX32iIEsRF...,2025-01-22,Flight Risk,false,6.118,671,2025-05-05 23:08:...,2025-05-07,2025-05-28,9,0,0,0
false,/3xKJ0nyUTlySmMBC...,"[27, 9648]",1232546,en,Until Dawn,one year after he...,70.4967,/6O9nkcmZBymDXtxO...,2025-04-23,Until Dawn,false,6.5,125,2025-05-05 23:08:...,2025-05-07,2025-05-28,10,0,0,0


In [82]:
value = processor.df.select("title")
print(value)

+--------------------+
|               title|
+--------------------+
|     the accountant |
|  death of a unicorn|
|          de lydl se|
|                rust|
|             amadeus|
|                    |
| the penguin lessons|
|         flight risk|
|          until dawn|
|                    |
|final destination...|
|   in the lost lands|
|   memoir of a snail|
|         last breath|
|     the last supper|
|                opus|
|popeye the slayer...|
|           black bag|
|                    |
|       thunderbolts |
+--------------------+
only showing top 20 rows



In [84]:
action_rows = processor.df.filter(F.col("id") == "29")

# Show them
action_rows.show()

+--------+-------+---+-----------------+--------------------+------------+-------+------+-------+-----+----------+------------+----------+--------------------+----------+----------+
|   title|film_id| id|original_language|            overview|release_date|revenue|budget|runtime|adult|popularity|vote_average|vote_count|      ingestion_time|begin_date|  end_date|
+--------+-------+---+-----------------+--------------------+------------+-------+------+-------+-----+----------+------------+----------+--------------------+----------+----------+
|베테랑 2| 995926| 29|               ko|the veteran detec...|  2024-09-13|      0|     0|      0|false|   55.4906|       6.955|        55|2025-04-17 16:59:...|2025-04-23|2025-05-14|
+--------+-------+---+-----------------+--------------------+------------+-------+------+-------+-----+----------+------------+----------+--------------------+----------+----------+

